# This notebook is to help automatically import parquet schema to hive

Below is import of all needed dependencies. And in this sell you should pass path where parquet files located. 

In [1]:
# import pandas as pd
# import v3io_frames as v3f
import os

# Set path where parquet files located. They can be nested in folder. 
filepath = os.path.join('/examples/parquet_example')
print(filepath)

/examples/parquet_example


Here is creating of spark context with hive support.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Import parquet schema to hive").config("hive.metastore.uris", "thrift://hive:9083").enableHiveSupport().getOrCreate()

Define function below for getting sql script needed for creating table in hive using dataframe types as columns to table

In [3]:
def getCreateTableScript(databaseName, tableName, df):
    cols = df.dtypes
    createScript = "CREATE EXTERNAL TABLE " + databaseName + "." + tableName + "("
    colArray = []
    for colName, colType in cols:
        colArray.append(colName.replace(" ", "_") + " " + colType)
    createColsScript =   ", ".join(colArray )
    
    script = createScript + createColsScript + ") STORED AS PARQUET LOCATION '" + tableName + "'"
    print(script)
    return script
    

In [4]:
#define main function for creating table where arqument 'path' is path to parquet files 
def createTable(databaseName, tableName, path): 
    df = spark.read.parquet(path)
    sqlScript = getCreateTableScript(databaseName, tableName, df)
    spark.sql(sqlScript)

## Here is an example where you provide a path to a folder with parquet files and their are uploaded

Write here name of database. Database will create if it doesn't exist.
In this sell code goes over all files and dirs in provided path and using them for creating table.
File should be ended with .parquet format
Directory (in which stored parquet files) should be started with "."

In [5]:
databaseName = "hive"
spark.sql("CREATE DATABASE IF NOT EXISTS " + databaseName)


for fileOrDir in os.listdir(filepath):
    if fileOrDir.endswith(".parquet") :
        createTable(databaseName, fileOrDir.split(".parquet")[0], filepath + fileOrDir)
    elif not fileOrDir.startswith(".") :
        createTable(databaseName, fileOrDir, filepath + fileOrDir + "/*")



AnalysisException: 'org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:Unable to create database path file:/User/spark-warehouse/dima.db, failed to create database dima);'

# Test how it works

In [ ]:
# test how the tables were saved
spark.sql("show tables in " + databaseName).show()

In [ ]:
# test how saving to table works
tableName = "example1"
spark.sql("select * from " + databaseName + "." + tableName)